In [1]:
%%writefile serve_flask.py
from flask import Flask, request, jsonify
import xgboost as xgb
import pandas as pd
import numpy as np
import os

app = Flask(__name__)

# Load the model at startup
MODEL_PATH = "model.bst"
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Missing {MODEL_PATH}")
bst = xgb.Booster()
bst.load_model(MODEL_PATH)

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()              # expect JSON array of dicts
    df = pd.DataFrame(data)
    dmat = xgb.DMatrix(df)
    preds_log = bst.predict(dmat)
    preds = np.expm1(preds_log)            # invert log1p
    return jsonify(predictions=preds.tolist())

if __name__ == "__main__":
    # for notebook demos, enable reloader off or on as you like
    app.run(host="0.0.0.0", port=8080, debug=True)

Writing serve_flask.py


In [ ]:
# !pip install flask xgboost pandas numpy        # only once, if needed
!python serve_flask.py

 * Serving Flask app 'serve_flask'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.255.255.2:8080
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 478-491-613
127.0.0.1 - - [22/Apr/2025 20:18:53] "POST /predict HTTP/1.1" 200 -


In [ ]:
!nohup python serve_flask.py &> flask.log &